In [1]:
import numpy as np
import random
import copy

In [2]:
actions = [0, 1, 2, 3]  # up, down, left, right
states = [i for i in range(0, 54)]
barrier_states = [7, 11, 16, 20, 25, 29, 41]
goal_state = 8

In [3]:
MAX_ITER = 5
gamma = 0.9  # discount factor

In [4]:
def return_Q(s, a, values):
    if (s in barrier_states) or (s == goal_state):
        return 0
    
    p = 0.8
    
    #-------- up ---------
    if a == 0:
        ns = s - 9   # up
        if (ns < 0) or (ns in barrier_states):
            t1 = p * (-1 + (gamma * values[s]))
        elif ns == goal_state:
            t1 = p * (1 + (gamma * values[ns]))
        else:
            t1 = p * (0 + (gamma * values[ns]))

        ns = s + 9   # down
        if (ns > 53) or (ns in barrier_states):
            t2 = (1-p) * (-1 + gamma * values[s])  
        else:
            t2 = (1-p) * (0 + gamma * values[ns])  
                        
    #-------- down ---------    
    if a == 1: 
        ns = s + 9   # down
        if (ns > 53) or (ns in barrier_states):
            t1 = p * (-1 + (gamma * values[s])) 
        else:
            t1 = p * (0 + (gamma * values[ns]))

        ns = s - 9    # up
        if (ns < 0) or (ns in barrier_states):
            t2 = (1-p) * (-1 + gamma * values[s]) 
        elif ns == goal_state:
            t2 = (1-p) * (1 + gamma * values[ns]) 
        else:
            t2 = (1-p) * (0 + gamma * values[ns]) 
            
    #-------- left ---------    
    if a == 2: 
        ns = s - 1   # left
        if (ns % 9 == 8) or (ns < 0) or (ns in barrier_states):
            t1 = p * (-1 + (gamma * values[s])) 
        else:
            t1 = p * (0 + (gamma * values[ns]))

        ns = s + 1   # right
        if (ns % 9 == 0) or (ns in barrier_states):
            t2 = (1-p) * (-1 + gamma * values[s])  
        elif ns == goal_state:
            t2 = (1-p) * (1 + gamma * values[ns]) 
        else:
            t2 = (1-p) * (0 + gamma * values[ns]) 
            
    #-------- right ---------    
    if a == 3: 
        ns = s + 1   # right
        if (ns % 9 == 0) or (ns in barrier_states):
            t1 = p * (-1 + (gamma * values[s])) 
        elif ns == goal_state:
            t1 = p * (1 + (gamma * values[ns]))
        else:
            t1 = p * (0 + (gamma * values[ns]))

        ns = s - 1   # left
        if (ns % 9 == 8) or (ns < 0) or (ns in barrier_states):
            t2 = (1-p) * (-1 + gamma * values[s]) 
        else:
            t2 = (1-p) * (0 + gamma * values[ns])
            
    return t1 + t2

## Policy Iteration

In [5]:
# Policy Evaluation

def policyEvaluation(policy, values):
    for i in range(MAX_ITER):  # assume fixed number of iterations for evaluation
        new_values = np.array([0.]*54)
        for s in states:
            a = policy[s]
            new_values[s] = return_Q(s, a, values)
        values = copy.deepcopy(new_values)
    return values

In [6]:
# Policy Improvement

def policyImprovement(values):
    new_policy = np.array([0]*54)
    
    for s in states:
        Q = [0.] * len(actions)
        for a in actions:
            Q[a] = return_Q(s, a, values)
        new_policy[s] = np.argmax(Q)
        
    return new_policy

In [7]:
# Policy Iteration

def policyIteration(policy, values):
    while True:
        current_policy = policy
        values = policyEvaluation(policy, values)
        policy = policyImprovement(values)
#         print(policy)
#         print(values)
        if (current_policy == policy).all():  # we find the optimal policy
            break
    return policy

In [8]:
random.seed(10)
init_values = np.array([0.]*54)
init_policy = np.array([random.randint(0, 3) for i in range(54)])   # select random initial actions for first policy

final_policy = policyIteration(init_policy, init_values)
print('Optimal policy:', final_policy)

Optimal policy: [3 3 3 3 3 3 1 0 0 1 1 0 1 1 1 1 0 0 1 1 0 1 3 3 1 0 0 0 1 0 1 3 3 3 3 0 0
 3 3 3 0 0 0 0 0 3 3 3 3 3 3 3 3 0]
